In [ ]:
import networkx as nx
import time
import rtsvg
rt = rtsvg.RACETrack()
from mds_at_scale import LandmarkMDSLayout, PivotMDSLayout
from linknode_graph_patterns import  LinkNodeGraphPatterns
_patterns_ = LinkNodeGraphPatterns()
_g_        = _patterns_.createPattern('stanford_facebook_networks')
_df_       = _patterns_.nxGraphToPolarsDataFrame(_g_)

# Create the layout & fill in the pos dictionary
t0 = time.time()
_pos_lmds_ = LandmarkMDSLayout(_g_).results()
t1 = time.time()
_pos_pmds_ = PivotMDSLayout(_g_).results()
t2 = time.time()

# previous (compared to the force directed implementation) LMDs: 0.17 | PFDL: 16.10
print(f"LMDs: {(t1-t0):.2f} | PMDS: {(t2-t1):.2f}") # LMDs: 0.08 | PMDS: 0.16

# Color the nodes
_node_colors_ = {}
community_i   = 0
for _community_ in nx.community.louvain_communities(_g_):
    community_i += 1
    for _node_ in _community_: _node_colors_[_node_] = rt.co_mgr.getColor(community_i)

# Display it
_params_ = {'df':_df_, 'relationships':[('fm','to')], 'node_color':_node_colors_, 'w':512, 'h':512}
rt.tile([rt.link(pos=_pos_lmds_, **_params_), rt.link(pos=_pos_pmds_, **_params_)])